In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import numpy as np
import matplotlib as plt
import pandas as pd
import sys  

The goal is to create your own strategy, optimize it with in-sample historical data, and then test it with out-of-sample historical data. The out-of-sample historical data cannot be the same as your training data.

## Strategy Implementation

In [ ]:
class myStrategy(bt.Strategy):
    '''
    Implementation of my strategy. Strategy parameters below, optimize them to get the 
    best possible implementation of my strategy given historical data. 
    '''
    params = (
        ('maperiod_long', 15),
        ('maperiod_short',5),
        ('cross_factor',1.0),
        ('printlog',False)
    )

    def log(self, txt, dt=None, doprint=False):
        '''
        Simple logging function. 
        '''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        #indicators
        self.sma_long = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod_long)
        self.sma_short = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod_short)
        self.cross_factor = self.params.cross_factor


    def notify_order(self, order):
        '''
        Keeps track of buy/sell executions.
        '''
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        '''
        Keeps track of completed trades, ie: one round-trip.
        '''
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        '''
        Actual strategy part of the trade.
        '''
        self.log('Open, %.2f, Close, %.2f' % (self.dataopen[0],self.dataclose[0]))
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       # Check if we are in the market
        if not self.position:
            
            # Check if small window MA multiplied by a factor is smaller than long window MA
            if self.sma_short[0]*self.cross_factor < self.sma_long[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
     
        else:
            if self.sma_short[0]*self.cross_factor >= self.sma_long[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        self.log('(MA Short Period %2d, MA Long Period %2d, Cross-Fact %.2f) Ending Value %.2f' %
                     (self.params.maperiod_short, self.params.maperiod_long, 
                      self.params.cross_factor, self.broker.getvalue()), doprint=True)
       

## Strategy Plot

In [ ]:
cerebro = bt.Cerebro()

cerebro.addstrategy(myStrategy)
cerebro.broker = bt.brokers.BackBroker(slip_perc=0.01,slip_out=True) 

fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
#datapath = os.path.join('../data/raw/demo/orcl-1995-2014.txt')

# Create a Data Feed
#data = bt.feeds.YahooFinanceCSVData(
#    dataname=datapath,
#    # Do not pass values before this date
#    fromdate=datetime.datetime(1995, 1, 1),
#    # Do not pass values before this date
#    todate=datetime.datetime(2000, 12, 31),
#    # Do not pass values after this date
#    reverse=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.broker.set_slippage_fixed(.01)
cerebro.broker.setcommission(commission=0.0)
cerebro.broker.setcash(10000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False,iplot=True)
plt.rcParams["figure.figsize"] = (30,20)


This class is a helper class to analyze methods. 

In [ ]:
class myAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.maperiod_long = self.strategy.params.maperiod_long
        self.maperiod_short = self.strategy.params.maperiod_short
        self.cross_factor = self.strategy.params.cross_factor
        
    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0

    def notify_trade(self, trade):
        self.num_trades += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        output = {
            'params': (self.maperiod_long, self.maperiod_short, self.cross_factor),
            'profit': self.final_val - self.init_cash,
            '#trades': self.num_trades
        }
        
        return output

## In-sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

cerebro.broker = bt.brokers.BackBroker(slip_perc=0.01,slip_out=True) 


# Add a strategy
strats = cerebro.optstrategy(
    myStrategy,
    maperiod_short=(5,6,7,8,9,10),
    maperiod_long=(30,35,40,45,50,55,60,65,70),
    cross_factor =(0.9,1.0,1.1))
    
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(myAnalyzer, _name='myAnalyzer')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')


cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.broker.setcash(10000.0)
cerebro.broker.set_slippage_fixed(.01)
cerebro.broker.setcommission(commission=0.0)

# Run over everything
results = cerebro.run(maxcpus=1)

In [ ]:
params  = []
num_trades = []
profit   = []
for res in results:
    r = res[0].analyzers.myAnalyzer.get_analysis()
    params.append(r['params'])
    num_trades.append(r['#trades'])
    profit.append(r['profit'])


prof_ind = np.argmax(profit)
best_params = params[prof_ind]
print("Best Parameters: ",best_params,"\nTotal Profit: ", profit[prof_ind], "\nNumber of Trades: ", num_trades[prof_ind])

In [ ]:
cerebro = bt.Cerebro()

cerebro.addstrategy(myStrategy,maperiod_long = 55,maperiod_short = 5, cross_factor = 1.0)


data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.broker.set_slippage_fixed(.01)
cerebro.broker.setcommission(commission=0.0)
cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result_in = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False,iplot=True)
plt.rcParams["figure.figsize"] = (40,20)

in_sample_sharpe=result_in[0].analyzers.mysharpe.get_analysis()

In [ ]:
cerebro = bt.Cerebro()

cerebro.addstrategy(myStrategy,maperiod_long = 55,maperiod_short = 5, cross_factor = 1.0)


data = bt.feeds.PandasData(dataname = df['2018-01-01':'2019-01-01'])
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.broker.set_slippage_fixed(.01)
cerebro.broker.setcommission(commission=0.0)
cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result_out = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False,iplot=True)
plt.rcParams["figure.figsize"] = (40,20)


out_sample_sharpe=result_out[0].analyzers.mysharpe.get_analysis()

In [ ]:
print("In sample sharpe{}".format(in_sample_sharpe))
print("Out of sample sharpe{}".format(out_sample_sharpe))